<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/basic_examples/more_careful_basin_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Looking more carefully at basin selection

The assumption for these notebooks is that you are working in a notable container.

So these steps are turned into comments using a `#` symbol before each line.

If you want to work on google colab you will need to delete the `#` symbols

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

In [ ]:
!pip install lsdviztools &> /dev/null

## Part 1: Get data

**First step, copy your Opentopography API key into a text file called "my_OT_api_key.txt" and make sure it is in the same file system as this notebook.**

If you don't know how to do that got to the `basic_topography` section and read lesson 1.

### Edit code block below for your site!

* change latitude and longitude of your `lower_left` and `upper_right` corners.
* Make sure your api filename is correct.



In [ ]:
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdbasemaptools as bmt

lower_left = [35.39069375614902, -121.13859639445643]
upper_right = [35.71181685305922, -120.88777897358574]

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "DEM"
source_name = "COP30"


My_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = lower_left,
                        upper_right_coordinates = upper_right,
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
My_DEM.print_parameters()
My_DEM.download_pythonic()

DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

**Important**: you should always cite your data source. For this data, you should cite:

*European Space Agency, Sinergise (2021). Copernicus Global Digital Elevation Model. Distributed by OpenTopography. https://doi.org/10.5069/G9028PQB. Accessed: 2023-09-26*

## Part 2: Get a hillshade: **you have to do this for the rest of the plotting to work**

We are going to get a hillshade since this is used in all of the plotting routines in `lsdviztools`.

*You only need to execute this code, you don't need to change anything!*

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This prints the image. It will also save a png file.

In [ ]:
DataDirectory = "./"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=True,
                                 size_format="geomorphology",dpi=400,
                                 out_fname_prefix = "hillshade_plot")

print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

**Next step: make sure the filename of your API key matches the one below.** Filenames are case sensitive. So if your file is "my_OT_API_key.txt" this will not work in the code block below (because api needs to be lower case).

**Then update the latitude and longitude of the lower left and upper right corner of your study area. You can get a latitude and longitude by right clicking on google maps.**

## Part 3: Extracting the network: effect of the `threshold_contributing_pixels` parameter

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"threshold_contributing_pixels" : "2000",
                    "print_channels_to_csv" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"big_contrib_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "DEM_COP30_UTM"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "elevation(m)",
                                          points_fname = "DEM_COP30big_contrib_UTM_CN.csv",
                                          scale_points = True, column_for_scaling = "Stream Order",
                                          scaled_data_in_log = False, max_point_size = 20, min_point_size = 2,
                                          cmap="Blues", save_fig=True, size_format="geomorphology")

print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"threshold_contributing_pixels" : "250",
                    "print_channels_to_csv" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"small_contrib_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "DEM_COP30_UTM"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "elevation(m)",
                                          points_fname = "DEM_COP30small_contrib_UTM_CN.csv",
                                          scale_points = True, column_for_scaling = "Stream Order",
                                          scaled_data_in_log = False, max_point_size = 20, min_point_size = 2,
                                          cmap="Blues", save_fig=True, size_format="geomorphology")

print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

## Picking basins

Now lets pick some basins.

I am going to pick some


1.   Near the coast
2.   Near tributaries



In [ ]:
# Import pandas library
import pandas as pd

data = [ [35.56123802423716, -121.09011375090577],
         [35.511569303371715, -121.02036041819895],
         [35.47757996179068, -121.00916111270496],
         [35.48588949854096, -120.99648569972402],
         [35.510766774527205, -120.96950347155934],
         [35.609461609083375, -121.0777103322082]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

I'm going to add a column just to make sure the points have a size

In [ ]:
df['C'] = 20
df.head(5)

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "DEM_COP30_UTM"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "elevation(m)",
                                          points_fname = "basin_outlets.csv",
                                          manual_size = 30,
                                          cmap="Blues", save_fig=True, size_format="geomorphology")

print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

# See where the basins come out with different contributing pixels

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_basin_raster" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "print_chi_data_maps" : "true",
                    "threshold_contributing_pixels" : "50",
                    "write_hillshade" : "true",
                    "extend_channel_to_node_before_receiver_junction" : "true",
                    "search_radius_nodes" : "5"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"small_contrib_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
#import lsdviztools.lsdmapwrappers as lsdmw
r_prefix = Dataset_prefix+"_"+source_name +"small_contrib_UTM"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=400, save_fig = True,
                             out_fname_prefix = "basins_plot")

print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_basin_raster" : "true",
                    "find_basins" : "true",
                    "print_chi_data_maps" : "true",
                    "threshold_contributing_pixels" : "750",
                    "write_hillshade" : "true",
                    "minimum_basin_size_pixels" : "100000",
                    "extend_channel_to_node_before_receiver_junction" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"UTM_algorithm_basins"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
#import lsdviztools.lsdmapwrappers as lsdmw
r_prefix = Dataset_prefix+"_"+source_name +"UTM_algorithm_basins"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=400, save_fig = True,
                             out_fname_prefix = "basins_plot")

print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))